In [13]:
import tensorflow as tf 
import numpy as np
from numpy import expand_dims
from numpy.random import randint
import matplotlib
from matplotlib import pyplot
from keras.models import Sequential
from keras.datasets.mnist import load_data
from keras.layers import Conv2D , Conv2DTranspose , Reshape,Flatten,Dense,ReLU,LeakyReLU,Dropout
from keras.optimizers import Adam
from numpy import vstack

In [14]:
def discriminator():
	model = Sequential()
	model.add(Conv2D(64 , (3,3) , strides = (2,2), padding = 'same',input_shape = (28,28,1) ))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dropout(0.4))

	model.add(Conv2D(64 , (3,3) , strides = (2,2),padding= 'same' ))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dropout(0.4))
	model.add(Flatten())
	model.add(Dense(1,activation= 'sigmoid'))
	optimus = Adam(lr=0.0002, beta_1=0.5)
	model.compile(optimizer = optimus,loss = 'binary_crossentropy',metrics =['accuracy'])

	return model

In [15]:

def generator(latent_dim = 100):
	model =Sequential()
	n_nodes = 128*7*7
	model.add(Dense(n_nodes, input_dim = latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	#model.add(Dropout(0.3))

	model.add(Reshape((7,7,128)))

	model.add(Conv2DTranspose(128 , (4,4),strides =(2,2), padding= 'same'))
	model.add(LeakyReLU(alpha=0.2))
	#model.add(Dropout(0.3))

	
	model.add(Conv2DTranspose(128 , (4,4),strides =(2,2), padding= 'same'))
	model.add(LeakyReLU(alpha=0.2))
	#model.add(Dropout(0.3))

	model.add(Conv2D(1,(7,7),activation= 'sigmoid' ,padding = 'same'))
	
	return model


In [16]:

def gan(g_model ,d_model):

	d_model.trainable = False
	model = Sequential()
	model.add(g_model)
	model.add(d_model)
    
	optimus = Adam(lr=0.0002, beta_1=0.5)
	model.compile(optimizer = optimus,loss = 'binary_crossentropy')

	return model

In [17]:
def load_true_data():

	(x_train, _),(_, _) = load_data() # loading data (28*28)
	x_train = expand_dims(x_train , axis =-1) # adding dim of channel (1)
	x_train = x_train.astype('float32')

	x_train = (x_train)/255.0
	return x_train

In [18]:

def generate_real_examples(dataset , n_samples):
	index = randint(0, dataset.shape[0] , n_samples)
	x = dataset[index]
	y = np.ones((n_samples,1))
	return x,y

In [19]:

def generate_latent_point(latent_dim , n_samples):

	inputs = np.random.randn(latent_dim * n_samples)
	inputs = inputs.reshape(n_samples,latent_dim)

	return inputs

In [20]:

def generate_fake_samples(g_model,latent_dim,n_samples):
	x_inp = generate_latent_point(latent_dim, n_samples)
	x = g_model.predict(x_inp)
	y = np.zeros((n_samples,1))

	return x,y

In [21]:

def save_plots(examples,epoch,n = 10):

	for i in range(n * n):
		pyplot.subplot(n,n,1+i)
		pyplot.axis('off')
		pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
	filename = 'generated_plot_e%03d.png' % (epoch+1)
	pyplot.savefig(filename)
	pyplot.close()

In [22]:
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples = 100):

	x_real, y_real = generate_real_examples(dataset,n_samples)
	_, acc_real = d_model.evaluate(x_real,y_real ,verbose = 0)

	x_fake, y_fake = generate_fake_samples(g_model,latent_dim,n_samples)
	_, acc_fake = d_model.evaluate(x_fake,y_fake ,verbose = 0)

	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	save_plots(x_fake,epoch)
	filename = 'generator_model_%03d.h5' % (epoch+1)
	g_model.save(filename)


In [23]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=6, n_batch=256):
	batch = int(n_batch/2)
	batch_per_epoch = int(dataset.shape[0] / n_batch)

	for i in range (n_epochs):
		for j in range(batch_per_epoch):

			x_real,y_real = generate_real_examples(dataset,batch)

			x_fake,y_fake = generate_fake_samples(g_model,latent_dim,batch)
			x,y = vstack((x_real,x_fake)),vstack((y_real,y_fake))
			d_loss,_ = d_model.train_on_batch(x,y)

			x_gan = generate_latent_point(latent_dim,n_batch)
			y_gan = np.ones((n_batch,1))

			g_loss = gan_model.train_on_batch(x_gan,y_gan)
			print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, batch_per_epoch, d_loss, g_loss))
		
		if (i+1)%3 ==0 :
			summarize_performance(i,g_model,d_model,dataset,latent_dim)

In [24]:

latent_dim = 100
d_model = discriminator()
g_model =generator(latent_dim)
gan_model = gan(g_model,d_model)

d_model.summary()
print("_______________________________________________________________________________________________________________")
g_model.summary()
dataset = load_true_data()
train(g_model,d_model,gan_model,dataset,latent_dim)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 14, 14, 64)        640       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3136)             

C:\Users\samya\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


>1, 2/234, d=0.701, g=0.673
>1, 3/234, d=0.695, g=0.695
>1, 4/234, d=0.685, g=0.718
>1, 5/234, d=0.673, g=0.735
>1, 6/234, d=0.663, g=0.764
>1, 7/234, d=0.655, g=0.781
>1, 8/234, d=0.652, g=0.798
>1, 9/234, d=0.642, g=0.814
>1, 10/234, d=0.634, g=0.835
>1, 11/234, d=0.629, g=0.848
>1, 12/234, d=0.620, g=0.859
>1, 13/234, d=0.626, g=0.862
>1, 14/234, d=0.622, g=0.848
>1, 15/234, d=0.623, g=0.821
>1, 16/234, d=0.627, g=0.793
>1, 17/234, d=0.638, g=0.763
>1, 18/234, d=0.641, g=0.741
>1, 19/234, d=0.639, g=0.723
>1, 20/234, d=0.637, g=0.710
>1, 21/234, d=0.632, g=0.703
>1, 22/234, d=0.627, g=0.699
>1, 23/234, d=0.620, g=0.696
>1, 24/234, d=0.608, g=0.695
>1, 25/234, d=0.602, g=0.695
>1, 26/234, d=0.595, g=0.695
>1, 27/234, d=0.581, g=0.694
>1, 28/234, d=0.573, g=0.695
>1, 29/234, d=0.562, g=0.696
>1, 30/234, d=0.559, g=0.696
>1, 31/234, d=0.543, g=0.697
>1, 32/234, d=0.534, g=0.697
>1, 33/234, d=0.526, g=0.698
>1, 34/234, d=0.519, g=0.699
>1, 35/234, d=0.508, g=0.700
>1, 36/234, d=0.492, g

>2, 47/234, d=0.611, g=0.874
>2, 48/234, d=0.608, g=0.874
>2, 49/234, d=0.616, g=0.891
>2, 50/234, d=0.595, g=0.890
>2, 51/234, d=0.571, g=0.936
>2, 52/234, d=0.573, g=0.899
>2, 53/234, d=0.584, g=0.892
>2, 54/234, d=0.619, g=0.878
>2, 55/234, d=0.606, g=0.860
>2, 56/234, d=0.597, g=0.866
>2, 57/234, d=0.611, g=0.822
>2, 58/234, d=0.580, g=0.809
>2, 59/234, d=0.594, g=0.823
>2, 60/234, d=0.629, g=0.789
>2, 61/234, d=0.596, g=0.792
>2, 62/234, d=0.607, g=0.785
>2, 63/234, d=0.620, g=0.793
>2, 64/234, d=0.615, g=0.804
>2, 65/234, d=0.630, g=0.807
>2, 66/234, d=0.621, g=0.803
>2, 67/234, d=0.645, g=0.794
>2, 68/234, d=0.614, g=0.803
>2, 69/234, d=0.646, g=0.791
>2, 70/234, d=0.652, g=0.786
>2, 71/234, d=0.647, g=0.782
>2, 72/234, d=0.659, g=0.760
>2, 73/234, d=0.674, g=0.727
>2, 74/234, d=0.685, g=0.737
>2, 75/234, d=0.698, g=0.747
>2, 76/234, d=0.699, g=0.728
>2, 77/234, d=0.740, g=0.727
>2, 78/234, d=0.738, g=0.720
>2, 79/234, d=0.750, g=0.704
>2, 80/234, d=0.732, g=0.679
>2, 81/234, d=

>3, 92/234, d=0.687, g=0.743
>3, 93/234, d=0.680, g=0.740
>3, 94/234, d=0.688, g=0.720
>3, 95/234, d=0.678, g=0.718
>3, 96/234, d=0.681, g=0.715
>3, 97/234, d=0.680, g=0.734
>3, 98/234, d=0.669, g=0.737
>3, 99/234, d=0.688, g=0.751
>3, 100/234, d=0.678, g=0.748
>3, 101/234, d=0.675, g=0.760
>3, 102/234, d=0.663, g=0.752
>3, 103/234, d=0.663, g=0.750
>3, 104/234, d=0.664, g=0.766
>3, 105/234, d=0.671, g=0.753
>3, 106/234, d=0.669, g=0.730
>3, 107/234, d=0.664, g=0.756
>3, 108/234, d=0.673, g=0.758
>3, 109/234, d=0.684, g=0.755
>3, 110/234, d=0.656, g=0.774
>3, 111/234, d=0.649, g=0.751
>3, 112/234, d=0.676, g=0.771
>3, 113/234, d=0.653, g=0.763
>3, 114/234, d=0.668, g=0.770
>3, 115/234, d=0.651, g=0.766
>3, 116/234, d=0.662, g=0.753
>3, 117/234, d=0.663, g=0.749
>3, 118/234, d=0.668, g=0.733
>3, 119/234, d=0.680, g=0.783
>3, 120/234, d=0.655, g=0.774
>3, 121/234, d=0.652, g=0.793
>3, 122/234, d=0.663, g=0.756
>3, 123/234, d=0.673, g=0.761
>3, 124/234, d=0.656, g=0.765
>3, 125/234, d=0.6

>4, 134/234, d=0.749, g=0.691
>4, 135/234, d=0.752, g=0.695
>4, 136/234, d=0.730, g=0.666
>4, 137/234, d=0.737, g=0.667
>4, 138/234, d=0.726, g=0.662
>4, 139/234, d=0.726, g=0.684
>4, 140/234, d=0.732, g=0.683
>4, 141/234, d=0.725, g=0.682
>4, 142/234, d=0.733, g=0.697
>4, 143/234, d=0.728, g=0.707
>4, 144/234, d=0.710, g=0.711
>4, 145/234, d=0.699, g=0.692
>4, 146/234, d=0.713, g=0.692
>4, 147/234, d=0.693, g=0.706
>4, 148/234, d=0.692, g=0.709
>4, 149/234, d=0.698, g=0.713
>4, 150/234, d=0.695, g=0.717
>4, 151/234, d=0.676, g=0.735
>4, 152/234, d=0.680, g=0.740
>4, 153/234, d=0.688, g=0.743
>4, 154/234, d=0.658, g=0.745
>4, 155/234, d=0.668, g=0.738
>4, 156/234, d=0.669, g=0.753
>4, 157/234, d=0.672, g=0.738
>4, 158/234, d=0.694, g=0.754
>4, 159/234, d=0.693, g=0.750
>4, 160/234, d=0.693, g=0.756
>4, 161/234, d=0.705, g=0.757
>4, 162/234, d=0.685, g=0.739
>4, 163/234, d=0.702, g=0.707
>4, 164/234, d=0.694, g=0.733
>4, 165/234, d=0.713, g=0.729
>4, 166/234, d=0.704, g=0.747
>4, 167/23

>5, 177/234, d=0.663, g=0.788
>5, 178/234, d=0.666, g=0.778
>5, 179/234, d=0.665, g=0.771
>5, 180/234, d=0.680, g=0.746
>5, 181/234, d=0.684, g=0.735
>5, 182/234, d=0.685, g=0.729
>5, 183/234, d=0.677, g=0.715
>5, 184/234, d=0.683, g=0.693
>5, 185/234, d=0.682, g=0.698
>5, 186/234, d=0.685, g=0.681
>5, 187/234, d=0.679, g=0.685
>5, 188/234, d=0.692, g=0.675
>5, 189/234, d=0.705, g=0.668
>5, 190/234, d=0.702, g=0.675
>5, 191/234, d=0.691, g=0.689
>5, 192/234, d=0.688, g=0.692
>5, 193/234, d=0.708, g=0.686
>5, 194/234, d=0.692, g=0.686
>5, 195/234, d=0.711, g=0.703
>5, 196/234, d=0.698, g=0.690
>5, 197/234, d=0.718, g=0.696
>5, 198/234, d=0.714, g=0.684
>5, 199/234, d=0.712, g=0.701
>5, 200/234, d=0.721, g=0.695
>5, 201/234, d=0.720, g=0.716
>5, 202/234, d=0.710, g=0.682
>5, 203/234, d=0.735, g=0.693
>5, 204/234, d=0.711, g=0.691
>5, 205/234, d=0.734, g=0.687
>5, 206/234, d=0.725, g=0.681
>5, 207/234, d=0.720, g=0.677
>5, 208/234, d=0.724, g=0.668
>5, 209/234, d=0.740, g=0.686
>5, 210/23

>6, 220/234, d=0.695, g=0.725
>6, 221/234, d=0.689, g=0.718
>6, 222/234, d=0.699, g=0.725
>6, 223/234, d=0.688, g=0.719
>6, 224/234, d=0.695, g=0.722
>6, 225/234, d=0.690, g=0.717
>6, 226/234, d=0.694, g=0.710
>6, 227/234, d=0.687, g=0.720
>6, 228/234, d=0.682, g=0.712
>6, 229/234, d=0.686, g=0.715
>6, 230/234, d=0.690, g=0.700
>6, 231/234, d=0.693, g=0.710
>6, 232/234, d=0.687, g=0.707
>6, 233/234, d=0.675, g=0.716
>6, 234/234, d=0.680, g=0.714
>Accuracy real: 52%, fake: 66%


In [ ]:
# things that a better result 
# 1. Set Optimizers parameters As set above
# 2. Dropout (0.4)